# Latent Dirichlet Allocation for sentiment words segmentation

In [4]:
import os
import sys
import re
import warnings
from collections import Counter, OrderedDict
from typing import List, Optional

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
from IPython.display import clear_output


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


tqdm_notebook.pandas()
warnings.filterwarnings('ignore')
np.random.seed(42)

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize.casual import casual_tokenize
from wordcloud import WordCloud, STOPWORDS


nltk.download('stopwords')
RUSSIAN_STOPWORDS = set(stopwords.words("russian"))
clear_output()

In [6]:
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation, PCA, TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import silhouette_score

In [18]:
image_captions_df = pd.read_csv("../visual-sentiment/image_sentiment_data_with_captions.csv")
text_posts_df = pd.read_csv("../ru-text-sentiment/data/lemmatize_labeled_rusentiment_text_mystem.csv")

In [19]:
image_captions_df.head()

,image_path,sentiment,caption,clean_caption
0,data/preprocess_images/positive/104505632.jpg,positive,"Женщина разговаривает по мобильному телефону, ...",женщина разговаривать мобильный телефон держат...
1,data/preprocess_images/positive/1059249846.jpg,positive,Женщина сидит на скамейке в парке с зонтиком.,женщина сидеть скамейка парк зонтик
2,data/preprocess_images/positive/108501238.jpg,positive,Женщина в очках и с длинными волосами держит м...,женщина очки длинный волос держать мобильный т...
3,data/preprocess_images/positive/1089046474.jpg,positive,Вид на город с высоты птичьего полета.,вид город высота птичий полет
4,data/preprocess_images/positive/1096822187.jpg,positive,Мужчина и женщина позируют фотографу.,мужчина женщина позировать фотограф


In [20]:
text_posts_df["sentiment"] = text_posts_df["sentiment"].replace({-1: "negative", 0: "neutral", 1: "positive"})
text_posts_df.head()

,text,sentiment
0,украина это часть русь искусственно отделять к...,negative
1,говорить относительно небольшой коррупция обра...,negative
2,свой поляк сделать русский понятно литовский п...,neutral
3,фильм зритель любить диковинный соседка пятый ...,neutral
4,государство платить посмотреть денежный поток ...,negative


In [21]:
image_captions_df = image_captions_df[["clean_caption", "sentiment"]].rename({"clean_caption": "text"}, axis=1)
image_captions_df.head()

,text,sentiment
0,женщина разговаривать мобильный телефон держат...,positive
1,женщина сидеть скамейка парк зонтик,positive
2,женщина очки длинный волос держать мобильный т...,positive
3,вид город высота птичий полет,positive
4,мужчина женщина позировать фотограф,positive


In [35]:
posts_for_ldia= pd.concat([text_posts_df, image_captions_df])

In [36]:
posts_for_ldia = posts_for_ldia.dropna(subset=["text"]).sample(frac=1).reset_index(drop=True)

In [37]:
posts_for_ldia.head()

,text,sentiment
0,помнить миссия проходить тревор вскрывать моги...,positive
1,шутить финансовый план внатура очень хуево,negative
2,ужное вернуть автобус ленин оба сторона усугуб...,negative
3,мнэээ давать прошивка свой диск,negative
4,ахах дчипил мо вчителек анга мовь слово ладушка,positive


In [38]:
posts_for_ldia.shape

(360449, 2)

In [51]:
counter = CountVectorizer(tokenizer=casual_tokenize, max_features=1024)

In [52]:
%%time
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=posts_for_ldia["text"]).toarray())

CPU times: total: 58.6 s
Wall time: 58.8 s


In [53]:
columns_nums_ldia, terms_ldia = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys(),)))
bow_docs.columns = terms_ldia

In [54]:
bow_docs.shape

(360449, 1024)